## Let's make semi-auto pipeline
#### because i don't know why but i cannot get access to databricks free datasets..

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# 1. routes (only made input folder in s3)
input_path = "s3a://wj-spark-bucket/pipeline_input/"
checkpoint_path = "s3a://wj-spark-bucket/checkpoints/weather_pipeline_v1/"
output_path = "s3a://wj-spark-bucket/pipeline_output/"

In [0]:
# 2. schema! tell auto loader about the schema of the incoming data
user_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("score", DoubleType(), True)
])

In [0]:
# 3. Auto Loader 
'''
    설계도(user_schema)를 참고해서,
    지정된 S3 폴더(input_path)에 새로운 CSV 파일이 들어오는지 감시 카메라(cloudFiles)를 켜고 
    실시간으로 읽어라(readStream)
'''
streaming_df = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("header", "true") \
    .schema(user_schema) \
    .load(input_path)

In [0]:
# 4. filter those score equal or more than 80
final_df = streaming_df.filter(col("score") >= 80)

In [0]:
# 5. save it 
'''
    가공된 데이터(final_df)를 델타 형식(delta)으로, 
    기존 데이터 뒤에 덧붙여서(append) 저장해라. 
    이때 어디까지 했는지 메모(checkpoint)해가면서 하고, 
    지금 쌓여있는 파일만 처리한 뒤 종료(trigger)해라
'''
query = final_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_path) \
    .trigger(availableNow=True) \
    .start(output_path)

In [0]:
display(spark.read.format("delta").load(output_path))

### Spark's Lazy Evaluation and Execution Plan

##### when i ran through the above codes, spark looks like a perfectionist who doesn't start before the plan is all set up. XD


###### streaming_df: plan about how it will read data from s3
###### final_df(filter): plan about how to filter the data


##### Spark starts to execute when it sees writeStream.start(). It then finally starts to read data, filter data ... etc.

##### -> so when i call .start(), spark looks at the entire blueprint and realizes: oh. the factory is ready! let's do it!

##### that's why if I upload a new csv file to s3 and runs the start(), it does this bluprint - finds the new input, filter, output - and eventually gives the output. 

--

#### My another question was ->
##### 'Why can I see the result in full data? Because I uploaded data1.csv, data2.csv and data3.csv seperately, but everytime i run start() and display(), I can see the all users (over 80 score) in one table? How is this possible?'

##### I googled it why:
###### The files are divided to .parquet files and stored in output folder. And the folder has '_delta_log' which knows what files is for what parquet. 
###### And when display() runs, spark read '_delta_log' and read them all and show them all on the screen like it is a big table.